## Retrieve yellow taxi data from Oct-2018 to March-2019

In [52]:
# The following was modified from MAST30024 Tutorial 1
from urllib.request import urlretrieve
import os

output_relative_dir = '../data/'

# check if it exists as it makedir will raise an error if it does exist
if not os.path.exists(output_relative_dir):
    os.makedirs(output_relative_dir)
    
# now, for each type of data set we will need, we will create the paths
for target_dir in ('tlc_data', 'tute_data'): # taxi_zones should already exist
    if not os.path.exists(output_relative_dir + target_dir):
        os.makedirs(output_relative_dir + target_dir)

In [57]:
# The following was modified from MAST30024 Tutorial 1
URL_TEMPLATE = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_"#year-month.parquet

In [66]:
# The following was modiefied from MAST30024 Tutorial 1
# yellow taxi data for October, November, December 2018
YEAR = '2018'
MONTHS_2018 = range(10, 13)

tlc_output_dir = output_relative_dir + 'raw/tlc_data'

for month in MONTHS_2018:
    # 0-fill i.e 1 -> 01, 2 -> 02, etc
    month = str(month).zfill(2) 
    print(f"Begin month {month}")
    
    # generate url
    url = f'{URL_TEMPLATE}{YEAR}-{month}.parquet'
    # generate output location and filename
    output_dir = f"{tlc_output_dir}/{YEAR}-{month}.parquet"
    # download
    urlretrieve(url, output_dir) 
    
    print(f"Completed month {month}")

Begin month 10
Completed month 10
Begin month 11
Completed month 11
Begin month 12
Completed month 12


In [67]:
# The following was modiefied from MAST30024 Tutorial 1
# yellow taxi data for January, February, March 2019
YEAR = '2019'
MONTHS_2019 = range(1, 4)

tlc_output_dir = output_relative_dir + 'raw/tlc_data'

for month in MONTHS_2019:
    # 0-fill i.e 1 -> 01, 2 -> 02, etc
    month = str(month).zfill(2) 
    print(f"Begin month {month}")
    
    # generate url
    url = f'{URL_TEMPLATE}{YEAR}-{month}.parquet'
    # generate output location and filename
    output_dir = f"{tlc_output_dir}/{YEAR}-{month}.parquet"
    # download
    urlretrieve(url, output_dir) 
    
    print(f"Completed month {month}")

Begin month 01
Completed month 01
Begin month 02
Completed month 02
Begin month 03
Completed month 03


In [89]:
from pyspark.sql import SparkSession

# Create a spark session
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .getOrCreate()
)


In [90]:
# total number of records for yellow taxis October-2018 to March-2019

sdf_all = spark.read.parquet('../data/raw/tlc_data/')
sdf_all.count()

47798251

## Pre-process yellow taxi data

In [91]:
sdf_all.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: integer (nullable = true)
 |-- airport_fee: integer (nullable = true)



In [112]:
# Filter data and remove outliers - where passenger count <=0 and trip distance <=0 miles or >=246.9 miles

from pyspark.sql import functions as F

required_cols = ('tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 
                 'RatecodeID', 'PULocationID', 'DOLocationID', 'payment_type', 'tip_amount', 'total_amount')
processed_yellow_taxi_data = sdf_all.select(*required_cols).where(
    (F.col('passenger_count') > 0) 
    & (F.col('trip_distance') > 0)
    & (F.col('trip_distance') < 246.9)
    & (F.col('PULocationID') < 264)
    & (F.col('DOLocationID') < 264)
    & (F.col('payment_type') == 1)
)

processed_yellow_taxi_data.count()


32737776

In [101]:
from pyspark.sql import functions as F
required_cols = ('VendorID','tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 
                 'RatecodeID', 'PULocationID', 'DOLocationID', 'payment_type', 'tip_amount', 'total_amount')
sdf_all.select(*required_cols).where(
    (F.col('VendorID') == NA)
)

NameError: name 'NA' is not defined

In [ ]:
# convert csv files to parquet file format

import pandas as pd

df = pd.read_csv('../data/raw/oct.csv')
df.to_parquet('../data/raw/oct.parquet')
df.head()